In [1]:
import numpy as np, humanfriendly as hf
import time
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error

In [2]:
def get_error(model, Xtest, ytest):
    y_pred = model.predict(Xtest)
    return np.sqrt(mean_squared_error(ytest, y_pred)),model.__class__.__name__

In [3]:
def see_time(note,start):
    end = time.perf_counter()
    elapsed = end - start
    print (note,hf.format_timespan(elapsed, detailed=True))

In [4]:
def get_cross(model, data, target, groups=10):
    return cross_val_score(model, data, target, cv=groups,scoring='neg_mean_squared_error')

In [5]:
X = np.load('data/X_tips.npy')
y = np.load('data/y_tips.npy')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [6]:
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.fit_transform(X_test)

In [7]:
lasso = Lasso(random_state=0, alpha=0.1)
lasso.fit(X_train_std, y_train)
rmse, name = get_error(lasso, X_test_std, y_test)
print (name + '(rmse):',rmse)

Lasso(rmse): 0.8785977911833893


In [8]:
alpha_lasso = [1e-1]
params = {'alpha': alpha_lasso, 'positive': [True, False],'max_iter': [10, 50, 100]}
grid = GridSearchCV(lasso, params, cv=5, n_jobs=-1,verbose=1)
start = time.perf_counter()
grid.fit(X_train, y_train)
see_time('training time:',start)
bp = grid.best_params_
print (bp)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


training time: 1 second, 578 milliseconds, 538 microseconds and 100 nanoseconds
{'alpha': 0.1, 'max_iter': 10, 'positive': True}


[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:    1.4s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    1.5s finished


In [9]:
lasso = Lasso(**bp, random_state=0).fit(X_train_std, y_train)
rmse, name = get_error(lasso, X_test_std, y_test)
print (name + '(rmse):',rmse)

Lasso(rmse): 0.8781319871042924


In [10]:
start = time.perf_counter()
scores = get_cross(lasso, X, y)
see_time('cross-validation rmse:',start)
rmse = np.sqrt(np.mean(scores) * -1)
print (rmse)

cross-validation rmse: 14 milliseconds, 883 microseconds and 600 nanoseconds
1.0379804468729155
